### __import packages__

In [ ]:
from Covid19 import MaskAreaDetector
import cv2
import time
import matplotlib.pyplot as plt

### __Create mask area detector object for image preprocessing__  
※ before creating object, these files below should be within same directory as `Facial mask area detecting simulator.ipynb`  
- haarcascade_eye.xml  
- haarcascade_eye_tree_eyeglasses.xml  
- haarcascade_frontface.xml  
- haarcascade_lefteye_2splits.xml  
---
Location of files : [https://github.com/opencv/opencv/tree/master/data/haarcascades](https://github.com/opencv/opencv/tree/master/data/haarcascades)  

In [ ]:
detector = MaskAreaDetector()

### __Load pretrained model__  
Location of files : [https://drive.google.com/drive/folders/1KazLH5eTck8fWfENTty76Ytss3Rgi3LY?usp=sharing](https://drive.google.com/drive/folders/1KazLH5eTck8fWfENTty76Ytss3Rgi3LY?usp=sharing)

In [ ]:
from keras.models import load_model
model_path='your model path'
my_model=load_model(model_path+'model file name')

### __Main code__

In [ ]:
import os
img_path ='your image path'

files = os.listdir(img_path)
image_files = [file for file in files if file.endswith(".jpg")] # or any other format like png
result_label = {0:'chin_mask', 1:'full_mask' , 2:'no_mask' , 3:'nose_mask'}
result_color = {0:(0,127,255), 1:(0, 255, 0) , 2:(0, 0, 255) , 3:(0,127,255)}

for file in image_files:
    print(f'{file} loading')
    test_image = cv2.imread(img_path+file)
    time.sleep(1.5)
    plt.figure()
    ax = plt.gca()
    ax.set_title('input raw image',fontsize=15)
    plt.imshow(test_image[:,:,::-1])
    plt.show()
    print('='*65)
    time.sleep(1.5)
    eyes = detector.advanced_eye_detect(test_image)
    if len(eyes)!=2:
        print('Initial Detection Failed : not human or invalid image')
        print('='*65)

    else:
        print('Initial Detection success')
        rotated_image = detector.get_rotated_image(test_image,eyes)
        rotated_eyes = detector.calculate_rotated_eyes(rotated_image)
        print('='*65)
        time.sleep(1.5)

        if len(rotated_eyes)!=2:
            print('Rotated Detection Failed : not human or invalid image')
            print('='*65)

        else:
            print('Rotated Detection success')
            print('='*65)
            time.sleep(1.5)
            current_mask_area = detector.extract_facial_mask_area(rotated_image,rotated_eyes)

            time.sleep(1.5)
            plt.figure()
            ax = plt.gca()
            ax.set_title('extracted mask area',fontsize=15)
            plt.imshow(current_mask_area[:,:,::-1])
            plt.show()

            print('='*65)
            time.sleep(1.5)
            
            current_mask_area = current_mask_area/255.0
            current_mask_area = cv2.resize(current_mask_area,dsize=(90, 120))
            test_in = current_mask_area.reshape((1,)+current_mask_area.shape)
            result = my_model.predict(test_in)
            print(result)
            print(f'predicted result : {result_label[result.argmax()]}')
            
            test_image = cv2.resize(test_image,(128,128))
            h,w,c = test_image.shape
            cv2.putText(test_image,result_label[result.argmax()].split('_')[0],(h//4,w//2),cv2.FONT_HERSHEY_COMPLEX,1,
                        result_color[result.argmax()],2,cv2.LINE_AA)
            plt.figure()
            ax = plt.gca()
            ax.set_title('Predicted result',fontsize=15)
            plt.imshow(test_image[:,:,::-1])
            plt.show()            
            print('='*65)
            time.sleep(1.5)
